<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/2DCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive
ls: cannot access '/content/drive/mydrive': No such file or directory


In [29]:
%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow

TRAIN_PATH = "train"
TEST_PATH ="test"

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

# **Dataset Loading**

Unzip del dataset (Temporaneamente da caricare da file)

In [3]:
import zipfile


zip_ref = zipfile.ZipFile('fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

Associazione path delle immagini - classe delle immagini

In [31]:
import os
class_dir=os.listdir(TRAIN_PATH+'/')
folder_names = class_dir
label_dict = {folder_names[i]:i for i in range(len(folder_names))}
num_classes=len(label_dict)
train_image_filenames = []
train_labels = []

# iterate through each folder and collect filenames and labels
for folder_name in folder_names:
    folder_path = os.path.join(TRAIN_PATH, folder_name)
    for filename in os.listdir(folder_path):
        train_image_filenames.append(os.path.join(TRAIN_PATH+"/"+folder_name, filename))
        train_labels.append(label_dict[folder_name])

# create pandas dataframe
train_df = pd.DataFrame({'filename': train_image_filenames, 'emotion': train_labels})

print(len(train_df))


test_image_filenames = []
test_labels = []
class_dir=os.listdir(TEST_PATH+'/')
for folder_name in folder_names:
    folder_path = os.path.join(TEST_PATH, folder_name)
    for filename in os.listdir(folder_path):
        test_image_filenames.append(os.path.join(TEST_PATH+"/"+folder_name, filename))
        test_labels.append(label_dict[folder_name])

test_df = pd.DataFrame({'filename': test_image_filenames, 'emotion': test_labels})

28709


Caricamento matrici delle immagini

In [32]:
train_df['img_as_matrix'] = train_df['filename'].apply(lambda path: cv2.imread(path))
test_df['img_as_matrix'] = test_df['filename'].apply(lambda path: cv2.imread(path))


Creazione del dataset, estendendo la classe dataset di PyTorch

In [33]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data.iloc[index]['img_as_matrix'] 
        label = self.data.iloc[index]['emotion'] 

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

Creazione dei Loader

In [34]:
train_df.drop(columns=["filename"], inplace=True)
test_df.drop(columns=["filename"], inplace=True)
train_df.at[0,"img_as_matrix"]
train_dataset = CustomDataset(train_df, transform=transforms.ToTensor())
test_dataset = CustomDataset(test_df, transform=transforms.ToTensor())

batch_size = 1024  

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Rete convolutiva**

In [35]:
def accuracy(preds, labels):
  _, predicted = torch.max(preds, 1) 
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc;


In [36]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.maxpool1 = nn.MaxPool2d(2)
        self.dropout1 = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3)
        self.batch_norm4 = nn.BatchNorm2d(128)
        self.maxpool2 = nn.MaxPool2d(2)
        self.dropout2 = nn.Dropout2d(0.25)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3)
        self.batch_norm5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3)
        self.batch_norm6 = nn.BatchNorm2d(256)
        self.maxpool3 = nn.MaxPool2d(2)
        self.dropout3 = nn.Dropout2d(0.25)
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(256*2*2, 256)
        self.batch_norm7 = nn.BatchNorm1d(256)
        self.dropout4 = nn.Dropout(0.5)
        self.dense2 = nn.Linear(256, 7)
        

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.batch_norm1(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.batch_norm2(x)
        x = self.maxpool1(x)
        x = self.dropout1(x)
        x = nn.functional.relu(self.conv3(x))
        x = self.batch_norm3(x)
        x = nn.functional.relu(self.conv4(x))
        x = self.batch_norm4(x)
        x = self.maxpool2(x)
        x = self.dropout2(x)
        x = nn.functional.relu(self.conv5(x))
        x = self.batch_norm5(x)
        x = nn.functional.relu(self.conv6(x))
        x = self.batch_norm6(x)
        x = self.maxpool3(x)
        x = self.dropout3(x)
        x = self.flatten(x)
        x = nn.functional.relu(self.dense1(x))
        x = self.batch_norm7(x)
        x = self.dropout4(x)
        x = self.dense2(x)
        
        return x



In [37]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [38]:
n_total_steps = len(train_loader)
num_epochs = 20
for epoch in range(num_epochs):

    running_loss = 0.0
    running_acc = 0.0

    for i, (images, labels) in enumerate(train_loader):
       
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)

      loss = criterion(outputs, labels)
      _, predicted = torch.max(outputs, 1)

      acc = accuracy(outputs, labels)

      optimizer.zero_grad()      
      loss.backward()            
      optimizer.step()  
      running_loss += loss.item()    
      running_acc += acc.item()

    print (f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {running_acc/len(train_loader):.4f}')

print('Finished Training')

PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [0/20], Loss: 2.1025, Acc: 17.6207
Epoch [1/20], Loss: 2.0070, Acc: 21.7931
Epoch [2/20], Loss: 1.9391, Acc: 24.7586
Epoch [3/20], Loss: 1.8772, Acc: 27.4483
Epoch [4/20], Loss: 1.8178, Acc: 30.8621
Epoch [5/20], Loss: 1.7517, Acc: 33.9655
Epoch [6/20], Loss: 1.7126, Acc: 35.3448
Epoch [7/20], Loss: 1.6799, Acc: 36.9655
Epoch [8/20], Loss: 1.6506, Acc: 38.3448
Epoch [9/20], Loss: 1.6092, Acc: 39.7241
Epoch [10/20], Loss: 1.5869, Acc: 40.2759
Epoch [11/20], Loss: 1.5664, Acc: 41.1724
Epoch [12/20], Loss: 1.5518, Acc: 41.5172
Epoch [13/20], Loss: 1.5356, Acc: 42.2414
Epoch [14/20], Loss: 1.5082, Acc: 43.7241
Epoch [15/20], Loss: 1.4992, Acc: 43.5862
Epoch [16/20], Loss: 1.4736, Acc: 44.8276
Epoch [17/20], Loss: 1.4664, Acc: 44.7586
Epoch [18/20], Loss: 1.4456, Acc: 45.5862
Epoch [19/20], Loss: 1.4370, Acc: 46.4138
Finished Training


In [39]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(7)]
    n_class_samples = [0 for i in range(7)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
       
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(labels.size(0)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(7):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {i}: {round(acc, 2)} %')

Accuracy of the network: 23.349122318194482 %
Accuracy of 0: 17.72 %
Accuracy of 1: 0.0 %
Accuracy of 2: 42.05 %
Accuracy of 3: 28.04 %
Accuracy of 4: 22.95 %
Accuracy of 5: 8.5 %
Accuracy of 6: 11.06 %
